# EMP500 metabolomics - Establish putative microbial secondary metabolites - FBMN

**Authors**: Louis Felix Nothias, Feb 2021

**Objective**: 
- We match putative spectral annotation against NPAtlas and MIBIG and recover metadata (prefix `NPA_` and `MIBIG_` respectively)

- We create new columns to indicate if an annotation is microbial (`is_microbial`). We differenciate the annotation level based on Metabolomics Standard Initiative standards. We also create new columns that store the identifiers from NPAtlas and MiBIG.

    - Level 2: GNPS spectral library match in regular and analaogue mode, prefix: `GNPS_LIB` and `GNPS_LIBA`
    - Level 3: DEREPLICATOR and DEREPLICATOR+, prefix:  `DEREP` and `DEREP+`
    - Level 4: SIRIUS/CSI:FingerID, prefix `CSI_`


- We propagate the microbial molecules using molecular network families (using `GNPS_componentindex`) and create a new column to indicate that these molecules are part of a putative microbial network, or the `GNPS_componentindex'


In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem

In [2]:
feature_metadata_table = 'annotations_2_1/FBMN_metabo_feature_metadata_filtered_consolidated.tsv'
features = pd.read_csv(feature_metadata_table,
                       sep='\t', low_memory=False)

In [3]:
npatlas = pd.read_csv('microbial_metabolite_database/NPAtlas_download.tsv', sep='\t')
new_names = [(i,'NPA_'+i) for i in npatlas.iloc[:, 0:].columns.values]
npatlas.rename(columns = dict(new_names), inplace=True)

In [4]:
# This prints the metadata columns per annotations using the respective prefix.
def show_metadata_tools(table, metadata_prefix):
    metadata = []
    for x in table.columns:
        if x.startswith(metadata_prefix):
            metadata.append(str(x))
    print(metadata)

### Generate the inchikey first block for all the annotation tools

In [5]:
#Make a new column for inchikey without stereo
def _prepare_inchikey(table, column):
    table[column+str('_no_stereo')] = (table[column]
                                .str.split('-').str[0])

In [6]:
#Input columns for inchikey
gnps_lib = 'GNPS_LIB_Consol_InChIKey'
gnps_liba = 'GNPS_LIBA_Consol_InChIKey'
derep = 'DEREP_Consol_InChIKey'
derepplus = 'DEREP+_Consol_InChIKey'
csi = 'CSI_InChIkey2D'

In [7]:
#Apply to the tables

    #NPAtlas
_prepare_inchikey(npatlas, 'NPA_compound_inchikey')

    #Feature annotation table
_prepare_inchikey(features, gnps_lib)
_prepare_inchikey(features, gnps_liba)
_prepare_inchikey(features, derep)
_prepare_inchikey(features, derepplus)
_prepare_inchikey(features, csi)

### Aggregate and match feature metadata with NPAtlas

In [8]:
# Aggregate and merge. Beautiful piece by Wout Bittremieux
def _aggregate_npatlas(table, npatlas, column, prefix):
    npatlas2 = npatlas
    npatlas2 = (npatlas2.groupby('NPA_compound_inchikey_no_stereo')
               [['NPA_npaid', 'NPA_compound_id', 'NPA_compound_names', 'NPA_origin_type',
                 'NPA_genus', 'NPA_origin_species', 'NPA_mibig_ids']]
               .agg(lambda values: '|'.join([str(v) for v in values]))
               .reset_index())
    new_names = [(i,str(prefix)+i) for i in npatlas2.iloc[:, 0:].columns.values]
    npatlas2.rename(columns = dict(new_names), inplace=True)
    _aggregate_npatlas.merged = pd.merge(table, npatlas2, left_on=column+'_no_stereo', right_on=str(prefix)+'NPA_compound_inchikey_no_stereo', how ='left')

In [9]:
# Apply to each annotation tool
_aggregate_npatlas(features, npatlas, gnps_lib,'GNPS_LIB_')
_aggregate_npatlas(_aggregate_npatlas.merged, npatlas, gnps_liba,'GNPS_LIBA_')
_aggregate_npatlas(_aggregate_npatlas.merged, npatlas, derep,'DEREP_')
_aggregate_npatlas(_aggregate_npatlas.merged, npatlas, derepplus,'DEREP+_')
_aggregate_npatlas(_aggregate_npatlas.merged, npatlas, csi,'CSI_')

## Aggregate and match with MIBIG

In [10]:
mibig = pd.read_csv('microbial_metabolite_database/mibig.csv', sep=',')
new_names = [(i,'MIBIG_'+i) for i in mibig.iloc[:, 0:].columns.values]
mibig.rename(columns = dict(new_names), inplace=True)

In [11]:
# Function to aggregate with MiBIG
def _aggregate_mibig(table, mibig, column, prefix):
    mibig2 = mibig
    mibig2 = (mibig2.groupby('MIBIG_no_stereo_inchikey')
               [['MIBIG_mibig_accession', 'MIBIG_organism_name','MIBIG_compound_name','MIBIG_ncbi_tax_id']]
               .agg(lambda values: '|'.join([str(v) for v in values]))
               .reset_index())
    new_names = [(i,str(prefix)+i) for i in mibig2.iloc[:, 0:].columns.values]
    mibig2.rename(columns = dict(new_names), inplace=True)
    _aggregate_mibig.merged = pd.merge(table, mibig2, left_on=column+'_no_stereo', right_on=str(prefix)+'MIBIG_no_stereo_inchikey', how ='left')

In [12]:
# Apply to each annotation tool
_aggregate_mibig(_aggregate_npatlas.merged, mibig, gnps_lib,'GNPS_LIB_')
_aggregate_mibig(_aggregate_mibig.merged, mibig, gnps_liba,'GNPS_LIBA_')
_aggregate_mibig(_aggregate_mibig.merged, mibig, derep,'DEREP_')
_aggregate_mibig(_aggregate_mibig.merged, mibig, derepplus,'DEREP+_')
_aggregate_mibig(_aggregate_mibig.merged, mibig, csi,'CSI_')

## We check at what MSI level the microbial annotation is

In [13]:
table = _aggregate_mibig.merged

In [14]:
level_2 = []
level_3 = []
level_4 = []
level_2_3 = []
level_2_3_4 = []

# We are making columns to indicate if the compound is microbial and the annotation level of the tool
for i, row in table.iterrows():
    #level_2
    if row['GNPS_LIB_NPA_compound_id'] is not np.nan:
        level_2.append('YES')
    elif row['GNPS_LIBA_NPA_compound_id'] is not np.nan:
        level_2.append('YES')
    elif row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        level_2.append('YES')
    elif row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        level_2.append('YES')
    else:
        level_2.append(np.nan)
    
    #level_3  
    if row['DEREP_NPA_compound_id'] is not np.nan:
        level_3.append('YES')
    elif row['DEREP+_NPA_compound_id'] is not np.nan:
        level_3.append('YES')
    elif row['DEREP_MIBIG_mibig_accession'] is not np.nan:
        level_3.append('YES')
    elif row['DEREP+_MIBIG_mibig_accession'] is not np.nan:
        level_3.append('YES')
    else:
        level_3.append(np.nan)
        
    #level_4     
    if row['CSI_NPA_compound_id'] is not np.nan:
        level_4.append('YES')
    elif row['CSI_MIBIG_mibig_accession'] is not np.nan:
        level_4.append('YES')
    else:
        level_4.append(np.nan)
        
    #level_2_3
    if row['GNPS_LIB_NPA_compound_id'] is not np.nan:
        level_2_3.append('YES')
    elif row['GNPS_LIBA_NPA_compound_id'] is not np.nan:
        level_2_3.append('YES')
    elif row['DEREP_NPA_compound_id'] is not np.nan:
        level_2_3.append('YES')
    elif row['DEREP+_NPA_compound_id'] is not np.nan:
        level_2_3.append('YES')
    elif row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        level_2_3.append('YES')
    elif row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        level_2_3.append('YES')
    elif row['DEREP_MIBIG_mibig_accession'] is not np.nan:
        level_2_3.append('YES')
    elif row['DEREP+_MIBIG_mibig_accession'] is not np.nan:
        level_2_3.append('YES')
    else:
        level_2_3.append(np.nan)
    
    #level_2_3_4
    if row['GNPS_LIB_NPA_compound_id'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['GNPS_LIBA_NPA_compound_id'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['DEREP_NPA_compound_id'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['DEREP+_NPA_compound_id'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['CSI_NPA_compound_id'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['DEREP_MIBIG_mibig_accession'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['DEREP+_MIBIG_mibig_accession'] is not np.nan:
        level_2_3_4.append('YES')
    elif row['CSI_MIBIG_mibig_accession'] is not np.nan:
        level_2_3_4.append('YES')
    else:
        level_2_3_4.append(np.nan)
        
table['is_microbial_level_2'] = level_2
table['is_microbial_level_3'] = level_3
table['is_microbial_level_4'] = level_4
table['is_microbial_level_2_3'] = level_2_3
table['is_microbial_level_2_3_4'] = level_2_3_4

In [15]:
print('level_2 = '+ str(table[(table['is_microbial_level_2'] == 'YES')].shape[0]))
print('level_3 = '+ str(table[(table['is_microbial_level_3'] == 'YES')].shape[0]))
print('level_4 = '+ str(table[(table['is_microbial_level_4'] == 'YES')].shape[0]))
print('level_2_3 = '+ str(table[(table['is_microbial_level_2_3'] == 'YES')].shape[0]))
print('level_2_3_4 = '+ str(table[(table['is_microbial_level_2_3_4'] == 'YES')].shape[0]))

level_2 = 869
level_3 = 1286
level_4 = 1604
level_2_3 = 2116
level_2_3_4 = 3546


### For microbial annotation, we make a column that store information about annotation and accession number


In [16]:
element_list = []
id_list = []

for i, row in table.iterrows():
    element = []
    identifier = []
    
    #Initiating 
    if row['GNPS_LIB_NPA_npaid'] is not np.nan:
        element = 'GNPS_LIB_NPA|'
        identifier = row['GNPS_LIB_NPA_npaid']+str('|')
    else:
        element = ''
        identifier = ''
    
    #NPAtlas
    if row['GNPS_LIBA_NPA_npaid'] is not np.nan:
        element += 'GNPS_LIBA_NPA|'
        identifier += row['GNPS_LIBA_NPA_npaid']+str('|')
    if row['DEREP_NPA_npaid'] is not np.nan:
        element += 'DEREP_NPA|'
        identifier += row['DEREP_NPA_npaid']+str('|')
    if row['DEREP+_NPA_npaid'] is not np.nan:
        element += 'DEREP+_NPA|'
        identifier += row['DEREP+_NPA_npaid']+str('|')
    if row['CSI_NPA_npaid'] is not np.nan:
        element += 'CSI_NPA|'
        identifier += row['CSI_NPA_npaid']+str('|')
    
    #MIBIG
    if row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        element += 'GNPS_LIB_MIBIG|'
        identifier += row['GNPS_LIB_MIBIG_mibig_accession']+str('|')
    if row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        element += 'GNPS_LIBA_MIBIG|'
        identifier += row['GNPS_LIBA_MIBIG_mibig_accession']+str('|')
    if row['DEREP_MIBIG_mibig_accession'] is not np.nan:
        element += 'DEREP_MIBIG|'
        identifier += row['DEREP_MIBIG_mibig_accession']+str('|')
    if row['DEREP+_MIBIG_mibig_accession'] is not np.nan:
        element += 'DEREP+_MIBIG|'
        identifier += row['DEREP+_MIBIG_mibig_accession']+str('|')
    if row['CSI_MIBIG_mibig_accession'] is not np.nan:
        element += 'CSI_MIBIG|'
        identifier += row['CSI_MIBIG_mibig_accession']+str('|')
    
    element_list.append(element)
    id_list.append(identifier)
    
table['is_microbial_tool'] = element_list 
table['is_microbial_tool_id'] = id_list

### We are propagating microbial annotations using molecular networking

In [17]:
# We are propagating microbial annotation
def _propagate_microbial_annotation(table,column):

    propagate_list = []
    propagate_column = []
    propagate_componentid = []

    for i, row in table.iterrows():
        if row[column] is not np.nan:
            if row['GNPS_componentindex'] is not -1:
                propagate_list.append(row['GNPS_componentindex'])

    for i, row in table.iterrows():
        if row['GNPS_componentindex'] in propagate_list:
            propagate_column.append('YES')
            propagate_componentid.append(row['GNPS_componentindex'])
        else:
            propagate_column.append(np.nan)
            propagate_componentid.append(np.nan)
            
    table[column+'_network'] = propagate_column
    table[column+'_networkid'] = propagate_componentid

In [18]:
# We propagate for each annotation level
_propagate_microbial_annotation(table,'is_microbial_level_2')
_propagate_microbial_annotation(table,'is_microbial_level_3')
_propagate_microbial_annotation(table,'is_microbial_level_4')
_propagate_microbial_annotation(table,'is_microbial_level_2_3')
_propagate_microbial_annotation(table,'is_microbial_level_2_3_4')

In [19]:
print('level_2_network = '+ str(table[(table['is_microbial_level_2_network'] == 'YES')].shape[0]))
print('level_3_network = '+ str(table[(table['is_microbial_level_3_network'] == 'YES')].shape[0]))
print('level_4_network = '+ str(table[(table['is_microbial_level_4_network'] == 'YES')].shape[0]))
print('level_2_3_network = '+ str(table[(table['is_microbial_level_2_3_network'] == 'YES')].shape[0]))
print('level_2_3_4_network = '+ str(table[(table['is_microbial_level_2_3_4_network'] == 'YES')].shape[0]))

level_2_network = 2784
level_3_network = 4410
level_4_network = 7245
level_2_3_network = 6588
level_2_3_4_network = 10771


### We write out the generated table

In [20]:
table_out = table.drop('Unnamed: 0', axis=1)
table_out.to_csv(feature_metadata_table[:-4]+'_is_microbial.tsv', sep='\t', index=False)